# WebForms Example
----------------
WebForms example automation allows you to create a webform. Users submit data through different field types (text field, number input, hidden input, dropdown choice, checkbox). 

Upon submission of the form, the automation processes the inputs and responds with JSON-formatted data.

##### 1. Import BitSwan libraries

In [ ]:
from bspump.http.web.server import *
from bspump.jupyter import *
import asyncio

#### 2. Create sample event for testing
We start with an example event so that we have something to play with. This event will demonstrate content which will be submitted via the webform. In the example, you can see differnt content for different types of fields.


In [ ]:
event = {
    "form": {
        "subject":      "Test Subject",
        "description":  "This is a readonly description",
        "number":       42,
        "hiddenValue":  "hidden_value",
        "lol":          "b",
        "checkbox":     True,
        "date":         "2025-06-26",            
        "timestamp":    "2025-06-26T14:30:00"   
    }
}

##### 3. Define the webform
We define the webform. Definition of each field is as follows:

`TypeOfField("NameOfField", special parameters)`

**TypeOfField**

Selecting type of the field, available types:
- `DateField` .. setting up date 
- `DateTimeField` .. setting up date and time
- `TextField` .. inserting text
- `IntField` .. inserting number 
- `ChoiceField` .. dropdown with choises (user selects one of the choises)
- `CheckboxField` .. checkbox

**NameOfField**

Name of the field under which the content can be accessed in your code. See Pipeline section below.

**Special parameters**

Each field can also contain special parameters. Available parameters:
- `display` .. Text (e.g. name of the field) displayed above the field. If the parameter is not present, NameOfField is used. Example: `DateField("date", display="Date")`
- `readonly` .. When `readonly=True` the content of the fied cannot be modified by the user.
- `default` .. Default value of the field
- `choices` .. **Only applicable for**  `ChoiceField`. Contains choices for the dropdown choice field. Example: `ChoiceField("letter", choices=["a","b","c"])`


In [ ]:
auto_pipeline(
    source=lambda app, pipeline: WebFormSource(app, pipeline, route="/", fields=[
        DateField("date", display="Date"),
        DateTimeField("timestamp", display="Date & Time"),  
        TextField("subject", display="foor"),
        TextField("description", readonly=True, default="laa"),
        IntField("number"),
        TextField("hiddenValue", hidden=True),
        ChoiceField("lol", choices=["a","b","c"]),
        CheckboxField("checkbox"),
    ]),
    sink  =lambda app, pipeline: WebSink(app,pipeline),
    name  ="WebServerPipeline"
)

#### 4. Pipeline section

Everything after this is run every time an event comes in. At run time (once deployed), the `event` variable is automatically set with the value of the event that comes from the source (in this case the webform).

The content of the form is accessible at `event["form"]["NameOfField]`.

We can do whatever transformations we please, and then, by setting `event["response"]` we set the output of the form. 
In order for the event output be in JSON format, we need to create the content of `event["response"]` as a Python dictionary, and perform `json.dumps` operation before sending it.

Example: `event["response"] = json.dumps{"number_times_five" = event["form"]["number"] * 5}`.

It is important to envelope the `event["response"]` block to `try-except` clause as show below, thnaks to that if an error occurs (such as missing fields), the form returns an error message.

In [ ]:
# print the form content which is set up in the sample event
print(event["form"])

In [ ]:
# perform calculation
number_times_five = event["form"]["number"] * 5

In [ ]:
# set up the response
try:
    event["response"] = json.dumps({
    "msg":       event["form"]["subject"],
    "checkbox":  event["form"]["checkbox"],
    "number_times_five":    number_times_five,
    "lol":       event["form"]["lol"],
    "date":      event["form"]["date"],
    "timestamp": event["form"]["timestamp"],
    })
    event["content_type"] = "text/plain"
except e:
    event["response"] = {"error", str(e)}

##### 5. Testing

**Running the automation cell by cell**

To test the functionality before deployment of the automation, you can simply run the automation cell by cell. The file which will be used is the file from sample event defined above. 


In [ ]:
# Print the output of the automation run with sample event
print(event["response"])

**Running the automation in BitSwan Editor**

To run the automation in BitSwan editor click on **Deploy automation**, the form will be accessible under the button **Open External URL**. For further explanation how the BitSwan Editor and automations work, please follow the **Development guide**, accesible under the **Help** button in the Editor. 